# Introduction to Pandas
You'll learn the basic concepts of Pandas.

This demo is a jupyter notebook, i.e. intended to be run step by step.

Author: Eric Einspänner
<br>
Contributor: Nastaran Takmilhomayouni

First version: 6th of July 2023


Copyright 2023 Clinic of Neuroradiology, Magdeburg, Germany

License: Apache-2.0

*All data used here are fictitious.*

## Table of contents
0. [Initial Set-Up for Google Colab](#initial-set-up-for-google-colab)
1. [Initial Set-Up (offline)](#initial-set-up-offline)
2. [Creating DataFrames from scratch](#creating-dataframes-from-scratch)
3. [Viewing your data](#viewing-your-data)
4. [Getting some informations about your data](#getting-some-informations-about-your-data)
5. [Handling duplicates](#handling-duplicates)
6. [Column cleanup](#column-cleanup)
7. [Manage missing values](#manage-missing-values)
   1. [Imputation](#imputation)
8. [Removing null values](#removing-null-values)
9.  [Understanding your variables](#understanding-your-variables)
10. [DataFrame slicing, selecting, extracting](#dataframe-slicing-selecting-extracting)
    1.  [By column](#by-column)
    2.  [By rows](#by-rows)
    3.  [Conditional selections](#conditional-selections)
11. [Plotting](#plotting)

## Initial Set-Up for Google Colab
<u> Execute these code blocks just in Google Colab! </u>

In [ ]:
!git clone https://github.com/University-Clinic-of-Neuroradiology/python-bootcamp.git

In [ ]:
import os
import sys
from google.colab import output
output.enable_custom_widget_manager()

sys.path.insert(0,'/content/python-bootcamp/notebooks/DataManagement')
os.chdir(sys.path[0])

In [ ]:
%pip install -q ipympl numpy matplotlib pandas seaborn

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

## Initial Set-Up (offline)

In [ ]:
# Initial imports etc
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

## --- Start notebook ---

## Creating DataFrames from scratch
There are many ways to create a DataFrame from scratch, but a great option is to just use a simple `dict`.

In [ ]:
data = {
    'sex': ['M', 'W', 'W', 'M'], 
    'age': [66, 56, 71, 62]
}

And then pass it to the pandas DataFrame constructor:

In [ ]:
patients = pd.DataFrame(data)
print(patients)

Each `(key, value)` item in data corresponds to a column in the resulting DataFrame.

The Index of this DataFrame was given to us on creation as the numbers 0-3, but we could also create our own when we initialize the DataFrame.

Let's have patient IDs as our index:

In [ ]:
patients = pd.DataFrame(data, index=['56171', '34167', '89143', '56124'])
print(patients)

**Attention:** We do not want to use integers as patient IDs, but strings!

So now we could locate a patient by using their ID:

In [ ]:
print(patients.loc['34167'])

## Viewing your data

In [ ]:
# Generate random patient IDs
patient_ids = ['54793', '39267', '69143', '12894', '64823', '38167', '89144', '56125', '34591', '44167']

# Generate random sex data
sex_data = np.random.choice(['M', 'W'], size=10)

# Generate random age data
age_data = np.random.randint(50, 81, size=10)

# Generate random aneurysm data
aneurysm_data = np.random.choice(['y', 'n'], size=10)

# Generate random aneurysm size data (when aneurysm is present)
aneurysm_size_data = [np.random.randint(5, 15) if aneurysm == 'y' else np.nan for aneurysm in aneurysm_data]

# Create DataFrame
df = pd.DataFrame({
    'patient_id': patient_ids,
    'Sex': sex_data,
    'age': age_data,
    'aneurysm': aneurysm_data,
    'Aneurysm Size [mm]': aneurysm_size_data
})

# Set Patient ID as index
df.set_index('patient_id', inplace=True)

The first thing to do when opening a new dataset is print out a few rows to keep as a visual reference. `.head()` outputs the first five rows of your DataFrame by default, but we could also pass a number as well: `df.head(7)` would output the top seven rows, for example.

In [ ]:
print(df.head())

To see the last five rows use `.tail()`.

## Getting some informations about your data

With `.info()` you get a bunch of informations:

In [ ]:
print(df.info())

`.info()` provides the essential details about your dataset, such as the number of rows and columns, the number of non-null values, what type of data is in each column, and how much memory your DataFrame is using.

**Notice:** in our dataset we have some obvious missing values in the `aneurysm_size_mm` column. We'll look at how to handle those in a bit.

Another fast and useful attribute is `.shape` (like for np.arrays), which outputs just a tuple of (rows, columns):

In [ ]:
print(df.shape)

## Handling duplicates

Our dataset does not have duplicate rows, but it is always important to verify you aren't aggregating duplicate rows.

To demonstrate, let's simply just double up our DataFrame by appending it to itself:

In [ ]:
temp_df = pd.concat([df, df])

print(temp_df.shape)

**Note:** We are capturing this copy in `temp` so we aren't working with the real data.

Now we can try dropping duplicates:

In [ ]:
temp_df = temp_df.drop_duplicates()

print(temp_df.shape)

## Column cleanup

Many times datasets will have verbose column names with symbols, upper and lowercase words, spaces, and typos. To make selecting data by column name easier we can spend a little time cleaning up their names.

Here's how to print the column names of our dataset:

In [ ]:
print(df.columns)

We can use the `.rename()` method to rename certain or all columns via a dict:

In [ ]:
df.rename(columns={
    'Sex': 'sex',
    'Aneurysm Size [mm]': 'aneurysm_size_mm'
    }, inplace=True)

print(df.columns)

## Manage missing values

When exploring data, you’ll most likely encounter missing or null values, which are essentially placeholders for non-existent values. Most commonly you'll see Python's `None` or NumPy's `np.nan`, each of which are handled differently in some situations.

There are two options in dealing with nulls:
- Get rid of rows or columns with nulls
- Replace nulls with non-null values, a technique known as **imputation**

Let's calculate to total number of nulls in each column of our dataset. The first step is to check which cells in our DataFrame are null:

In [ ]:
print(df.isnull())

Notice `isnull()` returns a DataFrame where each cell is either True or False depending on that cell's null status.

To count the number of nulls in each column we use an aggregate function for summing:

In [ ]:
print(df.isnull().sum())

## Removing null values

Remove nulls is pretty simple:

In [ ]:
temp_df = df.dropna()

print(temp_df.shape)

This operation will delete any row with at least a single null value, but it will return a new DataFrame without altering the original one.

#### Imputation

Imputation is a conventional feature engineering technique used to keep valuable data that have null values.

There may be instances where dropping every row with a null value removes too big a chunk from your dataset, so instead we can impute that null with another value, usually the mean or the median of that column.

**Of course it doesn't make any sense here, but we would like to show you briefly how it works!**

Let's look at imputing the missing values in the `aneurysm_size_mm` column. First we'll extract that column into its own variable. Using square brackets is the general way we select columns in a DataFrame.

In [ ]:
aneurysm_size = df['aneurysm_size_mm']

print(aneurysm_size.head())

We'll impute the missing values of `aneurysm_size_mm` using the mean. Here's the mean value:

In [ ]:
aneurysm_size_mean = aneurysm_size.mean()

print(aneurysm_size_mean)

With the mean, let's fill the nulls using fillna():

In [ ]:
temp_df = df.fillna(aneurysm_size_mean)

print(temp_df.isnull().sum())

**Imputing an entire column with the same value like this is a basic example. It would be a better idea to try a more granular imputation**

## Understanding your variables

Using `describe()` on an entire DataFrame we can get a summary of the distribution of continuous variables:

In [ ]:
print(df.describe())

`.describe()` can also be used on a single column:

In [ ]:
print(df['sex'].describe())

`.value_counts()` can tell us the frequency of all values in a column:

In [ ]:
print(df['aneurysm'].value_counts())

## DataFrame slicing, selecting, extracting

Another important function is the targeted selection of data from the DataFrame.

It's important to note that, although many methods are the same, DataFrames and Series have different attributes, so you'll need be sure to know which type you are working with or else you will receive attribute errors.

#### By column
You can extract a column using square brackets like this:

In [ ]:
age_col = df['age']

Now, check the type:

In [ ]:
print(type(age_col))

This will return a Series. To extract a column as a DataFrame, you need to pass a list of column names. In our case that's just a single column:

In [ ]:
age_col = df[['age']]

print(type(age_col))

Since it's just a list, adding another column name is easy:

In [ ]:
temp_df = df[['age', 'aneurysm']]

print(temp_df.head())

#### By rows
For rows, we have two options:
- `.loc` - locates by name
- `.iloc`- locates by numerical index
Remember that we are still indexed by patient ID, so to use `.loc` we give it the `patient_id`:

In [ ]:
pat_39267 = df.loc['39267']

print(pat_39267)

On the other hand, with `iloc` we give it the numerical index of the patient ID:

In [ ]:
pat_39267 = df.iloc[1]

print(pat_39267)

`loc` and `iloc` can be thought of as similar to Python list slicing. To show this even further, let's select multiple rows.

In [ ]:
pat_subset = df.loc[['39267', '12894']]

print(pat_subset)

pat_subset = df.iloc[[1, 3]]

print(pat_subset)

#### Conditional selections
We’ve gone over how to select columns and rows, but what if we want to make a conditional selection?

To do that, we take a column from the DataFrame and apply a Boolean condition to it. Here's an example of a Boolean condition:

In [ ]:
condition = (df['age'] > 60)

print(condition.head())

Similar to `isnull()`, this returns a Series of True and False values.

We want to filter out all patients that are older than 60, in other words, we don’t want the False ages. To return the rows where that condition is True we have to pass this operation into the DataFrame:

In [ ]:
temp_df = df[df['age'] > 60]

print(temp_df.head())

We can make some richer conditionals by using logical operators `|` for "or" and `&` for "and":

In [ ]:
temp_df = df[(df['age'] > 60) & (df['aneurysm'] == 'y')]

print(temp_df.head())

We need to make sure to group evaluations with parentheses so Python knows how to evaluate the conditional.

Using the `isin()` method we could make this more concise though:

In [ ]:
temp_df = df[df['aneurysm'].isin(['y'])]

print(temp_df.head())

## Plotting
Another great thing about pandas is that it integrates with Matplotlib, so you get the ability to plot directly off DataFrames and Series.

In [ ]:
df.plot(kind='scatter', x='age', y='aneurysm_size_mm', title='Aneurysm Size by Age', color='r')
plt.show()

By combining categorical and continuous data, we can create a Boxplot:

In [ ]:
df.boxplot(column='aneurysm_size_mm', by='aneurysm')
plt.show()

If we want to plot a simple Histogram based on a single column, we can call plot on a column:

In [ ]:
df[(df['aneurysm']=='y')].plot(kind='hist', y='age', title='Age distribution by positive aneurysm', color='g', bins=5, alpha=0.5, legend=False)
plt.show()